In [5]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm 

In [6]:
def max_accuracy(accuracies, diagonal = True):
    """
    Return the maximum accuracy for each session.
    """
    # average over folds
    avg = np.mean(accuracies, axis=1)

    acc_max = []
    for i in range(avg.shape[0]): # looping over sessions
        if diagonal:
            max_acc = np.max(avg[i].diagonal())
        else:
            max_acc = np.max(avg[i])
        acc_max.append(max_acc)
    
    return acc_max

## Linear model to investigates the effects of the decoding strategy given the variablity of the ERF's

In [13]:
lbo_file = 'accuracies_LDA_lbo'
lbo = np.load(f'accuracies/{lbo_file}.npy')

propb_file = 'accuracies_LDA_prop'
propb = np.load(f'accuracies/{propb_file}.npy')

lbo_acc = max_accuracy(lbo, diagonal = False)
propb_acc = max_accuracy(propb, diagonal = False)

# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)
# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)

### preparing design matrix ###
strategy = np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])
accuracy = np.array(lbo_acc + propb_acc)
std = np.concatenate([std_sessions, std_sessions])

# create the design matrix
design = pd.DataFrame({'strategy': strategy, 'std': std, 'accuracy': accuracy})

#y, X = dmatrices('accuracy ~ strategy + std + strategy:std', design)
y, X = dmatrices('accuracy ~ strategy:std', design)
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               accuracy   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                 -0.057
Method:                 Least Squares   F-statistic:                    0.2976
Date:                Thu, 15 Dec 2022   Prob (F-statistic):              0.595
Time:                        12:03:12   Log-Likelihood:                 38.671
No. Observations:                  14   AIC:                            -73.34
Df Residuals:                      12   BIC:                            -72.06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.6606      0.006    106.037      0.000       0.647       0.674
strategy:std    -0.1433      0.263     -0.545      0.595      -0.716       0.429
==============================================================================
Omnibus:                        7.389   Durbin-Watson:                   2.641
Prob(Omnibus):                  0.025   Jarque-Bera (JB):                4.272
Skew:                           1.313   Prob(JB):                        0.118
Kurtosis:                       3.651   Cond. No.                         59.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Difference in accuracy between the two strategies

In [16]:
# do a t-test
from scipy import stats
stats.ttest_rel(lbo_acc, propb_acc)

Ttest_relResult(statistic=0.9144182373099055, pvalue=0.3957572642389497)

In [12]:
for i in range(len(lbo_acc)):
    print(f'session {i+1}')
    print(f'lbo: {lbo_acc[i]}')
    print(f'prob: {propb_acc[i]}')
    print('Difference in accuracy: ', propb_acc[i]- lbo_acc[i])

session 1
lbo: 0.6474582183941314
prob: 0.6571200365721782
Difference in accuracy:  0.009661818178046788
session 2
lbo: 0.6660547050595204
prob: 0.6433623322871805
Difference in accuracy:  -0.02269237277233982
session 3
lbo: 0.6518690993309011
prob: 0.6590248883705563
Difference in accuracy:  0.0071557890396551915
session 4
lbo: 0.6955834028060757
prob: 0.688769695912553
Difference in accuracy:  -0.006813706893522675
session 5
lbo: 0.6479796737972298
prob: 0.6392038296800201
Difference in accuracy:  -0.008775844117209664
session 6
lbo: 0.6527325385599785
prob: 0.6490810152383935
Difference in accuracy:  -0.0036515233215850262
session 7
lbo: 0.6587599474135505
prob: 0.6577482557745716
Difference in accuracy:  -0.001011691638978851


## Cross decoding

In [14]:
file = 'cross_decoding_ncv_5'
cross = np.load(f'accuracies/{file}.npy')

In [15]:
# list of 7 empty lists
accuracies = []

for i in range(7):
    tmp = cross[i, :, :, :]
    for j in range(7):
        accuracies.append(np.max(tmp[j, :, :]))


# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)
# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)

std_test = std_sessions.tolist()*7
std_train = [std_sessions[i] for i in range(7) for j in range(7)]


# create the design matrix
design = pd.DataFrame({'std_test': std_test, 'std_train': std_train, 'accuracy': accuracies})

y, X = dmatrices('accuracy ~ std_train + std_test + std_train:std_test', design)
#y, X = dmatrices('accuracy ~ strategy:std', design)
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               accuracy   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     5.451
Date:                Thu, 15 Dec 2022   Prob (F-statistic):            0.00277
Time:                        12:06:29   Log-Likelihood:                 117.45
No. Observations:                  49   AIC:                            -226.9
Df Residuals:                      45   BIC:                            -219.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              4.5849      1.207      3.800      0.000       2.155       7.015
std_train           -114.6061     35.987     -3.185      0.003    -187.088     -42.124
std_test            -116.5259     35.987     -3.238      0.002    -189.008     -44.044
std_train:std_test  3353.8137   1073.317      3.125      0.003    1192.043    5515.585
==============================================================================
Omnibus:                       13.018   Durbin-Watson:                   1.860
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               13.559
Skew:                           1.131   Prob(JB):                      0.00114
Kurtosis:                       4.234   Cond. No.                     3.28e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.28e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# playground aka mess!!!

In [ ]:
props = np.load('./accuracies/accuracies_LDA_props.npy', allow_pickle=True).squeeze() # proportional session

for i in range(len(props)):
    plt.imshow(props[i], cmap='viridis')
    plt.show()